# [NTDS'18] tutorial 6: linear algebra for graphs and networkx
[ntds'18]: https://github.com/mdeff/ntds_2018

[Benjamin Ricaud](https://people.epfl.ch/benjamin.ricaud), [EPFL LTS2](https://lts2.epfl.ch)

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

## 1 The gradient, incidence and Laplacian matrices

### 1.1 Simple unweighted, undirected graph: the path graph

A first simple example to understand the definition of these matrices and their relationships.

In [ ]:
Gl = nx.path_graph(4)
nx.draw(Gl, with_labels=True)

In [ ]:
A = nx.adjacency_matrix(Gl)
A.todense() # numpy matrix

In [ ]:
S = nx.incidence_matrix(Gl, oriented=True)
S.todense()

In [ ]:
S.dot(S.T).todense()

In [ ]:
L = nx.laplacian_matrix(Gl)
L.todense()

$ SS^\top=L$
and here $\nabla = S$.

$S$ is defined only for the case of unweighted graphs.

### 1.2 General definition of the gradient

Let $\cal{V}$ be the vertex set and $\cal{E}$ the edge set.
The gradient $\nabla: \cal{V}\to \cal{E}$ is defined as follows:
$$(\nabla f)[i,j] = \sqrt{w_{ij}}(f[j]-f[i]),
$$
where $f\in\cal{V}$ and $w_{ij}$ is the edge weight between nodes $i$ and $j$.

Let us justify this definition.

### 1.3 Connection with the standard discrete setting

A one-dimensional regularly sampled signal can be seen as a signal on a path graph.

In [ ]:
# Let us build the path graph again
Gl = nx.path_graph(4)
pos = dict((n,(n,0)) for n in Gl.nodes())
nx.draw(Gl, pos, with_labels=True)

An example of a function and its gradient.

In [ ]:
# Example of a function on the nodes
f = [1, 1, 2, 1]
# Plot the function
plt.plot(f)
# plot the path graph
plt.plot([0, 1, 2, 3], [0, 0, 0, 0], 'k')  # black line
plt.scatter(*zip(*pos.values()), c='r', s=300)  # red dots
plt.grid()

The standard gradient is:
$$f'[1]=\nabla f[1] = \frac{f[2]-f[1]}{\delta x}.
$$
Let $w=1/\delta x^2$.  We have in matrix form:
$$\nabla =\left(\begin{matrix} -1& 1&0&0\\0 &-1&1&0\\0&0&-1&1\\0&0&0&-1 \end{matrix}\right)\times \frac{1}{dx} = 
\left(\begin{matrix} -\sqrt{w}& \sqrt{w}&0&0\\0 &-\sqrt{w}&\sqrt{w}&0\\0&0&-\sqrt{w}&\sqrt{w}\\0&0&0&-\sqrt{w} \end{matrix}\right).
$$
Here, it is a $4\times 4$ matrix (4 nodes).

$$\nabla f = 
\left(\begin{matrix} -\sqrt{w}& \sqrt{w}&0&0\\0 &-\sqrt{w}&\sqrt{w}&0\\0&0&-\sqrt{w}&\sqrt{w}\\0&0&0&-\sqrt{w} \end{matrix}\right)\left(\begin{matrix}f[0]\\f[1]\\f[2]\\f[3] \end{matrix}\right)=\left(\begin{matrix}f[1]-f[0]\\f[2]-f[1]\\f[3]-f[2]\\-f[3] \end{matrix}\right)\times \sqrt{w}.
$$
The transpose gives:
$$\nabla^\top f[1] = \frac{f[0]-f[1]}{\delta x}
$$

Problem:

* You have to define boundary conditions (periodic, infinite line,...), or have the same number of edges and nodes.

This is solved if the gradient is defined on the edges.

**Remark:** we could have $f'[1] = \frac{f[1]-f[0]}{\delta x}$, depending on the convention.

What about the Laplacian (seen as the second derivative)?
$$ L f[1]= \nabla^\top(\nabla f)[1] = \frac{f'[0]-f'[1]}{\delta x}=\frac{f[1]-f[0] - (f[2]-f[1])}{\delta x^2} = w(f[1]-f[0])+w(f[1]-f[2])
$$

### 1.4 A graph with weights

In [ ]:
# Let us change the weights of the path graph
Aw = A.copy()
Aw[0, 1] = 2
Aw[1, 0] = 2
Aw[1, 2] = 10
Aw[2, 1] = 10

In [ ]:
Gw = nx.from_numpy_array(Aw.todense())
nx.draw(Gw, with_labels=True)

In [ ]:
S = nx.incidence_matrix(Gw, oriented=True, weight='weight')
S.todense()

In [ ]:
S.dot(S.T).todense()

In [ ]:
L = nx.laplacian_matrix(Gw)
L.todense()

The definitions do not match any more in this case.

### 1.5 A directed graph example

In [ ]:
Gld = nx.path_graph(4, create_using=nx.DiGraph())

In [ ]:
nx.draw(Gld, with_labels=True)

In [ ]:
nx.incidence_matrix(Gld, oriented=True).todense()

In [ ]:
nx.adjacency_matrix(Gld).todense()

In [ ]:
nx.laplacian_matrix(Gld)
# Not implemented!

### 1.6 How to compute the gradient?

Let $N$ be the number of nodes and $E$ the number of edges.

Remarks:
* The weight matrix is a $N\times N$ matrix where the entries are edge weights.
* For each edge the gradient is $\nabla f [i,j] = \sqrt{w_{ij}}(f(j)-f(i))$.
* The gradient matrix is of size $E\times N$.

Construct the Gradient matrix by iterating over the row and columns of the adjacency matrix.

In [ ]:
# Let us load the path graph again
Gl = nx.path_graph(4)
A = nx.adj_matrix(Gl)
A = A.todense()
plt.figure(figsize=(2, 2))
nx.draw(Gl, with_labels=True)

In [ ]:
print('Weight matrix')
print(A)

Exercice: compute the gradient of this graph.

In [ ]:
# Let us compute the gradient
N = A.shape[0]  # number of nodes
E = np.sum(A>0)  # number of edges (non-zero entries of A)
gradient = np.zeros((E, N))
eij = 0  # edge index
for i in range(N):
    for j in range(N):
        wij = A[i, j]
        if wij > 0:
            print('Edge ({},{}) has index {} and weight {}.'.format(i, j, eij, wij))
            eij = eij + 1  # increment the edge index

**Remark:** it is twice the number of edges expected!

2 points of view:

* You discard half of the edges (choose a direction for each edge) or,

* You can see an undirected edge as a sum of 2 directed edges in opposite directions, but you have to divide by 2 for the Laplacian:
$$L = \frac{1}{2} \nabla^\top \nabla.$$

In [ ]:
L = nx.laplacian_matrix(Gl)

Compare the laplacian obtained from your gradient and the laplacian given by networkx.

In [ ]:
# Exercise

## 3 Eigenvectors and their visualization

### 3.1 The grid graph

In [ ]:
# Let us build a 2d grid graph
nb_rows, nb_cols = 5, 7
Gd = nx.grid_2d_graph(nb_cols, nb_rows, periodic=False)#True)
Ad = nx.adjacency_matrix(Gd)
# Choose regular positions
pos = dict((n, n) for n in Gd.nodes())
#nx.draw_networkx(Gd)
nx.draw(Gd, pos, with_labels=True)

Remark: networkx function `grid_2d_graph` label nodes with their 2d coordinates (i,j).

### 3.2 Function on a graph

A "Dirac delta" function is a function that is zero everywhere except at one point.

In [ ]:
nb_nodes = nb_rows * nb_cols
f = np.zeros((nb_nodes, 1))
peak_position = 5
f[peak_position] = 1

In [ ]:
nx.draw(Gd, pos,node_color=f.flatten())
plt.title('Dirac delta at position ' + str(peak_position))

In [ ]:
L = nx.laplacian_matrix(Gd)
eigval, eigvect = np.linalg.eigh(L.todense())

In [ ]:
u_k = np.array(eigvect[:, 3]).flatten()
nx.draw(Gd, pos, node_color=u_k)

In [ ]:
u_k

### 3.3 An irregular graph

Let us see what are the eigenvectors of the Barbell graph.
You can try other graphs in the [networkx list](https://networkx.github.io/documentation/stable/reference/generators.html).

For drawing the graph you can use [networkx graph layouts](https://networkx.github.io/documentation/stable/reference/drawing.html#module-networkx.drawing.layout). For example, `spectral_layout` gives the Laplacian eigenmaps.

In [ ]:
Gb = nx.barbell_graph(5, 3)
#nx.draw(Gb, pos=posGb, with_labels=True)
posGb = nx.spring_layout(Gb)
nx.draw(Gb, pos=posGb, with_labels=True)

In [ ]:
Lb = nx.laplacian_matrix(Gb)
eigval, eigvect = np.linalg.eigh(Lb.todense())

In [ ]:
u_k = np.array(eigvect[:, 4]).flatten()
nx.draw(Gb, pos=posGb, node_color=u_k)

Some eigenvectors are peaky!

In [ ]:
plt.plot(u_k)